# Laboratorio 5 - Parte 1

### Máquinas de Vectores de Soporte

### 2019-II

#### Profesor: Julián D. Arias Londoño
#### julian.ariasl@udea.edu.co


## Guía del laboratorio

En esta archivo va a encontrar tanto celdas de código cómo celdas de texto con las instrucciones para desarrollar el laboratorio.

Lea atentamente las instrucciones entregadas en las celdas de texto correspondientes y proceda con la solución de las preguntas planteadas.

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

#### Primer Integrante: Kevin Martínez Gallego
#### Segundo Integrante: Andrés Mauricio Álvarez Ortiz

In [1]:
from __future__ import division
import numpy as np
from numpy import random
from numpy import round
import math
import matplotlib.pyplot as plt
#Algunas advertencias que queremos evitar
import warnings
warnings.filterwarnings("always")
%matplotlib inline

## Ejercicio 1: Completar código con la librería

En este ejercicio deben resolver un problema de clasificación multi-clase usando una SVM. Teniendo en cuenta que la formulación original de la SVM sólo permite resolver problemas bi-clase, deben comparar los resultados obtenidos usando una estrategia **Uno vs Uno** con una estrategia **Uno vs El resto**.

Cargamos los datos:

In [3]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
digits = load_digits(n_class=4)

#--------- preprocesamiento--------------------
pca = PCA(0.99, whiten=True)
data = pca.fit_transform(digits.data)

#---------- Datos a usar ----------------------
X = data
Y = digits.target

In [4]:
print(X.shape)
print(Y.shape)
print(np.unique(Y, return_counts = True))

(720, 39)
(720,)
(array([0, 1, 2, 3]), array([178, 182, 177, 183], dtype=int64))


Consutar el manejo de la librería sklearn para entrenar un modelos SVM en: http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html#sklearn.svm.SVC. Para el caso multiclase la librería ya tiene implementada la estrategia **Uno vs Uno**, así que en este caso sólo deben llamar correctamente los métodos. Complete el siguiente código usando un clasificador basado en SVM:

In [5]:
import math
import numpy as np
from numpy import random
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold

def execute_SVC(kernel, regParam, gamma):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    porcentajeVectores = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento
        if(kernel == 'rbf'):
            model = SVC(C=regParam, kernel='rbf', gamma=gamma, decision_function_shape='ovo').fit(Xtrain, Ytrain)
        else:
            model = SVC(C=regParam, kernel=kernel, decision_function_shape='ovo').fit(Xtrain, Ytrain)

        #Validación
        Ytrain_pred = model.predict(Xtrain)
        Yest = model.predict(Xtest)

        porcentajeVectores[j] = np.sum(model.n_support_)/np.shape(Xtrain)[0]
        
        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1

    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    print('Porcentaje de vectores de soporte ', np.mean(porcentajeVectores))
    
    return(str(round(np.mean(EficienciaVal),5)), str(round(np.std(EficienciaVal),5)), str(round(np.mean(porcentajeVectores),5)))

In [6]:
execute_SVC('linear', 1, 0)

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
Porcentaje de vectores de soporte  0.26203839770590365


('0.97078', '0.01549', '0.26204')

## Ejercicio 2: Experimentos

Realice los experimientos necesarios para llenar la siguiente tabla:

In [27]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
#df_types["Eficiencia en validacion"][3] = "0.97077"
#df_types["Intervalo de confianza"][3] = "0.01548"
#df_types["% de Vectores de Soporte"][3] = "0.2620"
#df_types.sort_index(inplace=True)

index = 0
kernel = 'linear'
regParam = [0.001, 0.01, 0.1, 1, 10, 100]
gammaValues = [0.01, 0.1, 1]

for i in range(np.size(regParam)):
    eff, std_eff, sv = execute_SVC(kernel, regParam[i], -1)
    df_types["Eficiencia en validacion"][index] = eff
    df_types["Intervalo de confianza"][index] = std_eff
    df_types["% de Vectores de Soporte"][index] = sv
    index += 1

kernel = 'rbf'
for i in range(np.size(regParam)):
    C = regParam[i]
    for j in range(np.size(gammaValues)):
        eff, std_eff, sv = execute_SVC(kernel, C, gammaValues[j])
        df_types["Eficiencia en validacion"][index] = eff
        df_types["Intervalo de confianza"][index] = std_eff
        df_types["% de Vectores de Soporte"][index] = sv
        index += 1
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

Eficiencia durante el entrenamiento = 0.9226595902460473+-0.01797959849011947
Eficiencia durante la validación = 0.7960012912715104+-0.053296257064013926
Porcentaje de vectores de soporte  1.0
Eficiencia durante el entrenamiento = 0.9967600864350297+-0.0007994779941787222
Eficiencia durante la validación = 0.9706994383727887+-0.029264745294069673
Porcentaje de vectores de soporte  0.608322259431444
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9804458869759817+-0.01661113527781064
Porcentaje de vectores de soporte  0.28659534274928156
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
Porcentaje de vectores de soporte  0.26203839770590365
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9707771243055832+-0.015485734683194906
Porcentaje de vectores de soporte  0.26203839770590365
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [28]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                     0.796                 0.0533   
       0.010   0.00                    0.9707                0.02926   
       0.100   0.00                   0.98045                0.01661   
       1.000   0.00                   0.97078                0.01549   
       10.000  0.00                   0.97078                0.01549   
       100.000 0.00                   0.97078                0.01549   
rbf    0.001   0.01                   0.31562                0.10806   
               0.10                   0.27791                0.04275   
               1.00                   0.25417                0.00172   
       0.010   0.01                   0.31562                0.10806   
               0.10                   0.27791                0.04275   
               1.00                   0.25417                0.00172   
       0.100   0.01                   0.83203                0.03645   
               0.10                   0.27791                0.04275   
               1.00                   0.25417                0.00172   
       1.000   0.01                   0.98466                0.01152   
               0.10                   0.80855                0.04071   
               1.00                   0.25696                0.00649   
       10.000  0.01                   0.99025                0.00832   
               0.10                   0.81547                 0.0377   
               1.00                   0.25837                0.00603   
       100.000 0.01                   0.99025                0.00832   
               0.10                   0.81547                 0.0377   
               1.00                   0.25837                0.00603   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                       1.0  
       0.010   0.00                   0.60832  
       0.100   0.00                    0.2866  
       1.000   0.00                   0.26204  
       10.000  0.00                   0.26204  
       100.000 0.00                   0.26204  
rbf    0.001   0.01                       1.0  
               0.10                       1.0  
               1.00                   0.99954  
       0.010   0.01                       1.0  
               0.10                       1.0  
               1.00                       1.0  
       0.100   0.01                       1.0  
               0.10                       1.0  
               1.00                       1.0  
       1.000   0.01                   0.64305  
               0.10                   0.97267  
               1.00                       1.0  
       10.000  0.01                   0.57732  
               0.10                   0.97221  
               1.00                       1.0  
       100.000 0.01                   0.57732  
               0.10                   0.97221  
               1.00                       1.0

## Ejercicio 3: Completar código sin el uso de la librería

Cree dos funciones, una para entrenar un conjunto de modelos bajo la estrategia Uno vs el resto, usando como clasificador base una SVM. La segunda función debe usar el conjunto de modelos entrenados, y clasificar un conjunto de muestras de validación.

#### Nota: No se permite el uso de librerías para resolver este ejercicio

In [14]:
def TrainSVM_OnevsRest(X, Y, kernel, regParam, gamma):
    models = []
    for i in range(len(np.unique(Y))):        
        a = np.zeros(np.shape(Y)[0])        
        for j in range(len(Y)):            
            if(Y[j] == i):
                a[j] = 0
            else:                
                a[j] = 1
        if(kernel == 'rbf'):
            models.append(SVC(kernel='rbf', C=regParam, gamma=gamma, probability=True ,decision_function_shape='ovr').fit(X, a))
        else:
            models.append(SVC(kernel=kernel, C=regParam, probability=True ,decision_function_shape='ovr').fit(X, a))
       
    return models

# Evaluamos la probabilidad de cada muestra de pertencer a cada clase utilizando cada clasificador.
# Para cada muestra escogemos la clase con mayor probabilidad de pertenencia.
def ValidaSVM_OnevsRest(models, X):
    array = np.zeros((np.shape(X)[0], len(models)))
    for i in range(len(models)):
        values = models[i].predict_proba(X)
        array[:, i] = values[:,0]
    
    return(np.argmax(array, axis=1))


## Ejercicio 4: Entrenamiento

Use las funciones definidas en el punto anterior para llevar a cabo la prueba de simulación con el mismo conjunto de datos del punto 1.

In [32]:
def execute_SVC_one_vs_all(kernel, regParam, gamma):
    Folds = 4
    random.seed(19680801)
    EficienciaTrain = np.zeros(Folds)
    EficienciaVal = np.zeros(Folds)
    porcentajeVectores = np.zeros(Folds)
    skf = StratifiedKFold(n_splits=Folds)
    j = 0
    for train, test in skf.split(X, Y):
        Xtrain = X[train,:]
        Ytrain = Y[train]
        Xtest = X[test,:]
        Ytest = Y[test]

        #Normalizamos los datos
        scaler = preprocessing.StandardScaler().fit(Xtrain)
        Xtrain = scaler.transform(Xtrain)
        Xtest = scaler.transform(Xtest)

        #Haga el llamado a la función para crear y entrenar el modelo usando los datos de entrenamiento

        models = TrainSVM_OnevsRest(Xtrain, Ytrain, kernel, regParam, gamma)
        
        #Promedio del % de cantidad de SV para cada grupo de clasificadores one vs all
        totalModels = len(models)
        supportVectors = np.zeros(totalModels)
        for i in range(totalModels): 
            supportVectors[i] = np.sum(models[i].n_support_)/np.shape(Xtrain)[0]
        porcentajeVectores[j] = np.sum(supportVectors)/totalModels
        
        #Validación
        Ytrain_pred = ValidaSVM_OnevsRest(models, Xtrain)
        Yest = ValidaSVM_OnevsRest(models, Xtest)

        #Evaluamos las predicciones del modelo con los datos de test
        EficienciaTrain[j] = np.mean(Ytrain_pred.ravel() == Ytrain.ravel())
        EficienciaVal[j] = np.mean(Yest.ravel() == Ytest.ravel())
        j += 1

    print('Eficiencia durante el entrenamiento = ' + str(np.mean(EficienciaTrain)) + '+-' + str(np.std(EficienciaTrain)))
    print('Eficiencia durante la validación = ' + str(np.mean(EficienciaVal)) + '+-' + str(np.std(EficienciaVal)))
    print('Porcentaje de vectores de soporte ', np.mean(porcentajeVectores))
    
    return(str(round(np.mean(EficienciaVal),5)), str(round(np.std(EficienciaVal),5)), str(round(np.mean(porcentajeVectores),5)))

In [35]:
execute_SVC_one_vs_all('linear', 1, 0)

Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9777370884748747+-0.01637228483712346
Porcentaje de vectores de soporte  0.0692159188256291


('0.97774', '0.01637', '0.06922')

## Ejercicio 5: Experimentos

Realice los experimientos necesarios para llenar la siguiente tabla:

In [36]:
import pandas as pd
import qgrid
df_types = pd.DataFrame({
    'Kernel' : pd.Series(['lineal','lineal','lineal','lineal','lineal','lineal','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf','rbf']),
    'C' : pd.Series([0.001,0.01,0.1,1,10,100,0.001,0.001,0.001,0.01,0.01,0.01,0.1,0.1,0.1,1,1,1,10,10,10,100,100,100]),
    'gamma' : pd.Series([0,0,0,0,0,0,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1,0.01,0.1,1])})
df_types["Eficiencia en validacion"] = ""
df_types["Intervalo de confianza"] = ""
df_types["% de Vectores de Soporte"] = ""
df_types.set_index(['Kernel','C','gamma'], inplace=True)
#df_types["Eficiencia en validacion"][3] = "0.97633"
#df_types["Intervalo de confianza"][3] = "0.01837"
#df_types["% de Vectores de Soporte"][3] = "0.2778"
#df_types.sort_index(inplace=True)

index = 0
kernel = 'linear'
regParam = [0.001, 0.01, 0.1, 1, 10, 100]
gammaValues = [0.01, 0.1, 1]

for i in range(np.size(regParam)):
    eff, std_eff, sv = execute_SVC_one_vs_all(kernel, regParam[i], -1)
    df_types["Eficiencia en validacion"][index] = eff
    df_types["Intervalo de confianza"][index] = std_eff
    df_types["% de Vectores de Soporte"][index] = sv
    index += 1

kernel = 'rbf'
for i in range(np.size(regParam)):
    C = regParam[i]
    for j in range(np.size(gammaValues)):
        eff, std_eff, sv = execute_SVC_one_vs_all(kernel, C, gammaValues[j])
        df_types["Eficiencia en validacion"][index] = eff
        df_types["Intervalo de confianza"][index] = std_eff
        df_types["% de Vectores de Soporte"][index] = sv
        index += 1

qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget

Eficiencia durante el entrenamiento = 0.9930494921377366+-0.005145358195276409
Eficiencia durante la validación = 0.9525262864352779+-0.04168920867842125
Porcentaje de vectores de soporte  0.5251193884450822
Eficiencia durante el entrenamiento = 0.9981498469548391+-0.0013070466994085485
Eficiencia durante la validación = 0.9636693802249265+-0.03569161150591934
Porcentaje de vectores de soporte  0.295148272153567
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9749048207984206+-0.02269351268359228
Porcentaje de vectores de soporte  0.10393842211175236
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9777370884748747+-0.01637228483712346
Porcentaje de vectores de soporte  0.0692159188256291
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficiencia durante la validación = 0.9777370884748747+-0.01637228483712346
Porcentaje de vectores de soporte  0.0692159188256291
Eficiencia durante el entrenamiento = 1.0+-0.0
Eficienc

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

Ejecute la siguiente instrucción para dejar guardados en el notebook los resultados de las pruebas.

In [37]:
qgrid_widget.get_changed_df()

Eficiencia en validacion Intervalo de confianza  \
Kernel C       gamma                                                   
lineal 0.001   0.00                   0.95253                0.04169   
       0.010   0.00                   0.96367                0.03569   
       0.100   0.00                    0.9749                0.02269   
       1.000   0.00                   0.97774                0.01637   
       10.000  0.00                   0.97774                0.01637   
       100.000 0.00                   0.97774                0.01637   
rbf    0.001   0.01                   0.96507                0.03526   
               0.10                   0.90306                0.03786   
               1.00                   0.28348                0.01759   
       0.010   0.01                   0.96787                0.03387   
               0.10                   0.91827                0.03218   
               1.00                   0.25417                0.00172   
       0.100   0.01                   0.96787                0.03387   
               0.10                   0.91827                0.03218   
               1.00                   0.25417                0.00172   
       1.000   0.01                   0.98183                0.01883   
               0.10                   0.93216                0.03387   
               1.00                   0.25417                0.00172   
       10.000  0.01                   0.98884                0.01053   
               0.10                   0.94184                 0.0305   
               1.00                   0.25141                 0.0051   
       100.000 0.01                   0.98884                0.01053   
               0.10                   0.94184                 0.0305   
               1.00                   0.25141                 0.0051   

                     % de Vectores de Soporte  
Kernel C       gamma                           
lineal 0.001   0.00                   0.52512  
       0.010   0.00                   0.29515  
       0.100   0.00                   0.10394  
       1.000   0.00                   0.06922  
       10.000  0.00                   0.06922  
       100.000 0.00                   0.06922  
rbf    0.001   0.01                   0.50173  
               0.10                   0.62986  
               1.00                   0.66794  
       0.010   0.01                   0.54571  
               0.10                    0.9729  
               1.00                       1.0  
       0.100   0.01                   0.55358  
               0.10                   0.98008  
               1.00                       1.0  
       1.000   0.01                   0.34352  
               0.10                   0.97012  
               1.00                       1.0  
       10.000  0.01                   0.27754  
               0.10                   0.96688  
               1.00                       1.0  
       100.000 0.01                   0.27754  
               0.10                   0.96688  
               1.00                       1.0

*En las tablas el punto separa las cifras decimales, es decir 100.000 es 100 ó 1.000 es 1.